In [ ]:
import pandas as pd
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### 데이터 불러오기
temp = pd.read_pickle('/content/drive/MyDrive/DATA_전처리/자치구별 기온.pickle')
humid = pd.read_pickle('/content/drive/MyDrive/DATA_전처리/자치구별 습도.pickle')
rain = pd.read_pickle('/content/drive/MyDrive/DATA_전처리/자치구별 강수량.pickle')
people = pd.read_pickle('/content/drive/MyDrive/DATA_전처리/자치구별 인구 수.pickle')
people['자치구'] = people['자치구'].str.replace("\u3000","",regex = False)

### 누적 데이터 불러오기
rain_sum = pd.read_pickle('/content/drive/MyDrive/DATA_전처리/자치구별/누적 강수량.pickle')
humid_mean = pd.read_pickle('/content/drive/MyDrive/DATA_전처리/자치구별/평균 습도.pickle')
temp_total_diff = pd.read_pickle('/content/drive/MyDrive/DATA_전처리/자치구별/1년 기온차.pickle')
temp_monthly_diff = pd.read_pickle('/content/drive/MyDrive/DATA_전처리/자치구별/평균 일교차.pickle')

### 포트홀 불러오기 ###################################################################################
pothole = pd.read_pickle('/content/drive/MyDrive/DATA_전처리/pothole_final7.pickle') ##################
#######################################################################################################

In [ ]:
### 발생일 비어있는 경우를 위한...
rain_forna = rain_sum.groupby('자치구')['누적 강수량'].mean()
humid_forna = humid_mean.groupby('자치구')['평균 습도'].mean()
temp_total_diff_forna = temp_total_diff.groupby('자치구')['1년 기온차'].mean()
temp_monthly_diff_forna = temp_monthly_diff.groupby('자치구')['평균 일교차'].mean()

In [ ]:
# NaN은 그대로 두고, 나머지만 문자열로 변환
pothole['등록번호'] = pothole['등록번호'].apply(lambda x: int(x) if pd.notna(x) else pd.NA)

In [ ]:
pothole.head()

,자치구,행정동,경도,위도,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,포트홀 존재여부,평균_건물연령,등록번호,발생일
0,종로구,종로1가,126.979625,37.570080,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,1,29.5,20230104092201008,2022-12-05
1,종로구,종로1가,126.981618,37.570169,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,0,29.5,<NA>,NaT
2,종로구,종로1가,126.982593,37.570176,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,0,29.5,<NA>,NaT
3,종로구,종로1가,126.979760,37.570214,4.0,28631.0,1872.0,2959.0,33462.0,0.144373,1,29.5,20230717902759008,2023-06-17
4,종로구,종로1가,126.979760,37.570214,4.0,28631.0,1872.0,2959.0,33462.0,0.144373,1,29.5,20230717902759008,2023-06-17


In [ ]:
### 기록된 날짜 기준 한 달 전으로 '발생일' 생성
wrong = pothole['등록번호'].isna() | (~pothole['등록번호'].astype(str).str.startswith("2"))
pothole['발생일'] = pd.to_datetime(pothole['등록번호'].astype(str).where(~wrong).str[:8])
pothole['발생일'] = pothole['발생일'].apply(lambda x:x - pd.Timedelta(days = 30))
pothole.head(2)

,자치구,행정동,경도,위도,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,포트홀 존재여부,평균_건물연령,등록번호,발생일
0,종로구,종로1가,126.979625,37.570080,3.0,22007.0,1728.0,2352.0,26087.0,0.1564,1,29.5,20230104092201008,2022-12-05
1,종로구,종로1가,126.981618,37.570169,3.0,22007.0,1728.0,2352.0,26087.0,0.1564,0,29.5,<NA>,NaT


In [ ]:
### 인구 수
people_merged = pd.merge(pothole, people, on = '자치구', how = 'left')
people_merged.head(2)

,자치구,행정동,경도,위도,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,포트홀 존재여부,평균_건물연령,등록번호,발생일,인구 수
0,종로구,종로1가,126.979625,37.570080,3.0,22007.0,1728.0,2352.0,26087.0,0.1564,1,29.5,20230104092201008,2022-12-05,72414.058824
1,종로구,종로1가,126.981618,37.570169,3.0,22007.0,1728.0,2352.0,26087.0,0.1564,0,29.5,<NA>,NaT,72414.058824


In [ ]:
with_date = people_merged[people_merged['발생일'].notna()].copy()
without_date = people_merged[people_merged['발생일'].isna()].copy()

#### 발생일 존재

In [ ]:
with_date_1 = pd.merge(with_date, rain_sum, on = ['자치구', '발생일'], how = 'left')
with_date_2 = pd.merge(with_date_1, humid_mean, on = ['자치구', '발생일'], how = 'left')
with_date_3 = pd.merge(with_date_2, temp_total_diff, on = ['자치구', '발생일'], how = 'left')
with_date_4 = pd.merge(with_date_3, temp_monthly_diff, on = ['자치구', '발생일'], how = 'left')
with_date_4.head()

,자치구,행정동,경도,위도,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,포트홀 존재여부,평균_건물연령,등록번호,발생일,인구 수,누적 강수량,평균 습도,1년 기온차,평균 일교차
0,종로구,종로1가,126.979625,37.570080,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,1,29.5,20230104092201008,2022-12-05,72414.058824,3338.573374,64.588456,43.0,9.641935
1,종로구,종로1가,126.979760,37.570214,4.0,28631.0,1872.0,2959.0,33462.0,0.144373,1,29.5,20230717902759008,2023-06-17,72414.058824,3338.573374,65.783242,45.6,8.848387
2,종로구,종로1가,126.979760,37.570214,4.0,28631.0,1872.0,2959.0,33462.0,0.144373,1,29.5,20230717902759008,2023-06-17,72414.058824,3338.573374,65.783242,45.6,8.848387
3,종로구,종로1가,126.981416,37.570233,4.0,28631.0,1872.0,2959.0,33462.0,0.144373,1,29.5,20230525142827000,2023-04-25,72414.058824,3338.573374,64.795310,45.6,10.832258
4,종로구,종로1가,126.981416,37.570233,4.0,28631.0,1872.0,2959.0,33462.0,0.144373,1,29.5,20230525142827000,2023-04-25,72414.058824,3338.573374,64.795310,45.6,10.832258


#### 발생일 결측

In [ ]:
without_date_1 = pd.merge(without_date, rain_forna, on = ['자치구'], how = 'left')
without_date_2 = pd.merge(without_date_1, humid_forna, on = ['자치구'], how = 'left')
without_date_3 = pd.merge(without_date_2, temp_total_diff_forna, on = ['자치구'], how = 'left')
without_date_4 = pd.merge(without_date_3, temp_monthly_diff_forna, on = ['자치구'], how = 'left')
without_date_4.head()

,자치구,행정동,경도,위도,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,포트홀 존재여부,평균_건물연령,등록번호,발생일,인구 수,누적 강수량,평균 습도,1년 기온차,평균 일교차
0,종로구,종로1가,126.981618,37.570169,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,0,29.5,<NA>,NaT,72414.058824,2953.190932,64.807699,42.481657,8.790078
1,종로구,종로1가,126.982593,37.570176,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,0,29.5,<NA>,NaT,72414.058824,2953.190932,64.807699,42.481657,8.790078
2,종로구,종로1가,126.982593,37.570284,4.0,23172.0,2736.0,2149.0,28057.0,0.174110,0,29.5,<NA>,NaT,72414.058824,2953.190932,64.807699,42.481657,8.790078
3,종로구,종로1가,126.983343,37.570278,3.0,26544.0,2466.0,2851.0,31861.0,0.166881,0,29.5,<NA>,NaT,72414.058824,2953.190932,64.807699,42.481657,8.790078
4,종로구,종로1가,126.983342,37.570170,3.0,19442.0,1905.0,2291.0,23638.0,0.177511,0,29.5,<NA>,NaT,72414.058824,2953.190932,64.807699,42.481657,8.790078


#### 최종 저장

In [ ]:
pothole_merged = pd.concat([with_date_4, without_date_4])
pothole_merged.head()

,자치구,행정동,경도,위도,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,포트홀 존재여부,평균_건물연령,등록번호,발생일,인구 수,누적 강수량,평균 습도,1년 기온차,평균 일교차
0,종로구,종로1가,126.979625,37.570080,3.0,22007.0,1728.0,2352.0,26087.0,0.156400,1,29.5,20230104092201008,2022-12-05,72414.058824,3338.573374,64.588456,43.0,9.641935
1,종로구,종로1가,126.979760,37.570214,4.0,28631.0,1872.0,2959.0,33462.0,0.144373,1,29.5,20230717902759008,2023-06-17,72414.058824,3338.573374,65.783242,45.6,8.848387
2,종로구,종로1가,126.979760,37.570214,4.0,28631.0,1872.0,2959.0,33462.0,0.144373,1,29.5,20230717902759008,2023-06-17,72414.058824,3338.573374,65.783242,45.6,8.848387
3,종로구,종로1가,126.981416,37.570233,4.0,28631.0,1872.0,2959.0,33462.0,0.144373,1,29.5,20230525142827000,2023-04-25,72414.058824,3338.573374,64.795310,45.6,10.832258
4,종로구,종로1가,126.981416,37.570233,4.0,28631.0,1872.0,2959.0,33462.0,0.144373,1,29.5,20230525142827000,2023-04-25,72414.058824,3338.573374,64.795310,45.6,10.832258


In [ ]:
### 16개의 결측치, 자치구 확인 결과 서울이 아닌 것으로 판명..! => 제거합니당
pothole_merged.isna().sum()
pothole_merged = pothole_merged.dropna(subset = ['인구 수'])
pothole_merged.isna().sum()

,0
자치구,0
행정동,0
경도,0
위도,0
차선수,0
승용차,0
버스,0
트럭,0
총교통량,0
중대형차량 교통량,0


In [ ]:
pothole_merged[pothole_merged['1년 기온차'].isna()]

16

In [ ]:
pothole_merged.to_pickle('/content/drive/MyDrive/DATA_완성/준희 완료!.pickle')